In [1]:
from matplotlib import pyplot as plt
from random import randint
from utils import *
from utils_satellite import *
import pandas as pd
from PIL import Image
from numpy import linalg as LA
import math
import cv2
from image import *


In [11]:
from numpy import linalg as LA

def get_polys():
    xa, ya = project(q, r)
    red = [(xa[1] - xa[0]), (ya[1] - ya[0])]
    green = [(xa[2] - xa[0]), (ya[2] - ya[0])]
    blue = [(xa[3] - xa[0]), (ya[3] - ya[0])]

    a = [xa[0] + red[0]/2.8 + green[0]/3.1, ya[0] + red[1]/2.8 + green[1]/3.1, 'ro']
    b = [xa[0] + red[0]/2.8 - green[0]/3.1, ya[0] + red[1]/2.8 - green[1]/3.1, 'go']
    c = [xa[0] - red[0]/2.8 - green[0]/3.1, ya[0] - red[1]/2.8 - green[1]/3.1, 'bo']
    d = [xa[0] - red[0]/2.8 + green[0]/3.1, ya[0] - red[1]/2.8 + green[1]/3.1, 'yo']
    e = [xa[0] + red[0]/2.4 + green[0]/2.6 + blue[0]/3.3, ya[0] + red[1]/2.4 + green[1]/2.6 + blue[1]/3.3, 'ro']
    f = [xa[0] + red[0]/2.4 - green[0]/2.6 + blue[0]/3.3, ya[0] + red[1]/2.4 - green[1]/2.6 + blue[1]/3.3, 'go']
    g = [xa[0] - red[0]/2.4 - green[0]/2.6 + blue[0]/3.3, ya[0] - red[1]/2.4 - green[1]/2.6 + blue[1]/3.3, 'bo']
    h = [xa[0] - red[0]/2.4 + green[0]/2.6 + blue[0]/3.3, ya[0] - red[1]/2.4 + green[1]/2.6 + blue[1]/3.3, 'yo']

    vertices = [a,b,c,d,e,f,g,h]

    a = [xa[0] + red[0]/2.8 + green[0]/3.1, ya[0] + red[1]/2.8 + green[1]/3.1]
    b = [xa[0] + red[0]/2.8 - green[0]/3.1, ya[0] + red[1]/2.8 - green[1]/3.1]
    c = [xa[0] - red[0]/2.8 - green[0]/3.1, ya[0] - red[1]/2.8 - green[1]/3.1]
    d = [xa[0] - red[0]/2.8 + green[0]/3.1, ya[0] - red[1]/2.8 + green[1]/3.1]
    e = [xa[0] + red[0]/2.4 + green[0]/2.6 + blue[0]/3.3, ya[0] + red[1]/2.4 + green[1]/2.6 + blue[1]/3.3]
    f = [xa[0] + red[0]/2.4 - green[0]/2.6 + blue[0]/3.3, ya[0] + red[1]/2.4 - green[1]/2.6 + blue[1]/3.3]
    g = [xa[0] - red[0]/2.4 - green[0]/2.6 + blue[0]/3.3, ya[0] - red[1]/2.4 - green[1]/2.6 + blue[1]/3.3]
    h = [xa[0] - red[0]/2.4 + green[0]/2.6 + blue[0]/3.3, ya[0] - red[1]/2.4 + green[1]/2.6 + blue[1]/3.3]



    rects = [[e,f,g,h,e], [a,b,c,d,a], [a,b,e,f,a],[a,d,h,e,a],[c,b,f,g,c],[c,d,h,g,c]]

    centers = np.array([[xa[0], ya[0]]])
    return rects, centers

def get_qr(line):
    line = line.split(',')
    name = line[0]
    q = np.array([float(line[1]), float(line[2]), float(line[3]), float(line[4])])
    r = np.array([float(line[5]), float(line[6]), float(line[7])])
    return name,q,r

def zoom_on_satellite(img, q, r, k, target_shape):
    polys, centers = get_polys()

    width = img.shape[1]
    height = img.shape[0]
    minx, maxx, miny, maxy = width,0,height,0
    for poly in polys:
        for coord in poly:
            if coord[0] < minx:
                minx = coord[0]
            if coord[0] > maxx:
                maxx = coord[0]
            if coord[1] < miny:
                miny = coord[1]
            if coord[1] > maxy:
                maxy = coord[1]
    
    sat_width = maxx - minx
    sat_height = maxy - miny
    
    if sat_width > target_shape[1] / 1.5 or sat_height > target_shape[0] / 1.5:
        return img, k
    
    border = min(target_shape[1] / 5, target_shape[0] / 5)
    pleft  = max(0, minx - border)
    ptop   = max(0, miny - border)
    pright = -pleft
    pbot = -ptop
    sx = 1
    sy = 1

    rM2 = np.array([[1.0, 0.0, -pleft], [0.0, 1.0, -ptop], [0.0, 0.0, 1.0]])  # translation
    img = cv2.warpAffine(img, rM2[:2], (width, height))

    dx = float(pleft)/ width
    dy = float(ptop) / height
    
    cw = width
    ch = height

    # compute the new K according to transform
    trans = np.array([[1, 0, -cw * dx],
                      [0, 1, -ch * dy],
                      [0, 0, 1]])
    k = np.matmul(trans, rawk)  # new intrinsic
    
    return img[:target_shape[0], :target_shape[1]], k
    
    
    


In [16]:
fx = 0.0176  # focal length[m]
fy = 0.0176  # focal length[m]
nu = 1920  # number of horizontal[pixels]
nv = 1200  # number of vertical[pixels]
ppx = 5.86e-6  # horizontal pixel pitch[m / pixel]
ppy = ppx  # vertical pixel pitch[m / pixel]
fpx = fx / ppx  # horizontal focal length[pixels]
fpy = fy / ppy  # vertical focal length[pixels]
K = np.array([[fpx,   0, nu / 2],
     [0,   fpy, nv / 2],
     [0,     0,      1]])
rawk = K
with open('v5.csv') as example:
    for line in example:
        name, q, r = get_qr(line)
        img = cv2.imread('../competition/speed/images/test/' + name)
        new_img, new_k = zoom_on_satellite(img, q, r, K, [576, 928])
        pose_mat = np.hstack((np.transpose(quat2dcm(q)), np.expand_dims(r, 1)))
        if img.shape[0] != new_img.shape[0]:
            path = './zoomed/' + name
            save_img_with_label(new_img, pose_mat, new_k, path)
            #img = cv2.imread('../competition/speed/images/test/' + name)
            #path = './zoomed/orig' + name
            #save_img_with_label(img, pose_mat, K, path)

AttributeError: 'NoneType' object has no attribute 'shape'